## Import

Lorem ipsum dolor sit amet.

In [26]:
import scipy
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt
import os

In [24]:
def print_signal_info(data, sample_rate):
    print(f"Sample Rate: {sample_rate} Hz")
    print(f"Data Shape: {data.shape}")
    print(f"First 10 Samples: {data[:10]}")

def print_signal_graphs(data, sample_rate):
    # Use only the first channel if stereo
    if len(data.shape) == 2:
        data = data[:, 0]  

    time = np.linspace(0., len(data) / sample_rate, num=len(data))
    fig, axs = plt.subplots(1, 2, figsize=(18, 6))

    # Raw and envelope
    envelope = amplitude_envelope(data, sample_rate)

    axs[0].plot(time, data, label="Waveform")
    axs[0].plot(time, envelope, color='r', alpha=0.6, label="Amplitude Envelope")
    axs[0].set_xlabel("Time [s]")
    axs[0].set_ylabel("Amplitude")
    axs[0].set_title("Waveform and Amplitude Envelope")
    axs[0].legend()
    axs[0].grid()

    # Spectrogram
    axs[1].specgram(data, Fs=sample_rate, NFFT=1024, noverlap=512, cmap='inferno')
    axs[1].set_xlabel("Time [s]")
    axs[1].set_ylabel("Frequency [Hz]")
    axs[1].set_title("Spectrogram of Audio Signal")

    plt.tight_layout()
    plt.show()

def amplitude_envelope(signal, sample_rate, cutoff_freq=10):
    # Design a low-pass Butterworth filter
    nyquist = 0.5 * sample_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(1, normal_cutoff, btype='low', analog=False)
    # Apply the filter to the absolute value of the signal
    envelope = filtfilt(b, a, np.abs(signal))
    return envelope

## Nahrání signálu

Lorem ipsum dolor sit amet.

In [29]:
# NEZAPOMENOUT ŘEŠIT OBA KANÁLY

dir_celo = "Dataset\Cel"

files_celo = os.listdir(dir_celo)

wav_files = [file for file in files_celo if file.endswith('.wav')]

for i in range(5):
    file_path = os.path.join(dir_celo, wav_files[i])
    sample_rate, data = wavfile.read(file_path)
    print_signal_info(data, sample_rate)
    print_signal_graphs(data, sample_rate)

Sample Rate: 44100 Hz
Data Shape: (132299, 2)
First 10 Samples: [[-82  30]
 [-73  40]
 [-62  49]
 [-49  58]
 [-38  69]
 [-26  87]
 [-15 108]
 [ -8 129]
 [ -1 153]
 [  7 177]]
Sample Rate: 44100 Hz
Data Shape: (132299, 2)
First 10 Samples: [[ 30  46]
 [ 22  41]
 [ 17  36]
 [ 10  26]
 [ 10  14]
 [ 10  -1]
 [  2 -18]
 [ -1 -35]
 [  6 -46]
 [ 11 -51]]
Sample Rate: 44100 Hz
Data Shape: (132299, 2)
First 10 Samples: [[-247 -303]
 [-224 -294]
 [-200 -296]
 [-169 -307]
 [-126 -314]
 [ -69 -316]
 [ -13 -316]
 [  29 -311]
 [  58 -312]
 [  77 -322]]
Sample Rate: 44100 Hz
Data Shape: (132299, 2)
First 10 Samples: [[-329  363]
 [-363  268]
 [-423  174]
 [-494   32]
 [-544 -125]
 [-587 -238]
 [-641 -323]
 [-693 -393]
 [-730 -423]
 [-758 -412]]
Sample Rate: 44100 Hz
Data Shape: (132299, 2)
First 10 Samples: [[  28 -596]
 [ -40 -596]
 [ -69 -620]
 [-189 -646]
 [-362 -543]
 [-459 -314]
 [-372  -71]
 [-129   53]
 [  94   61]
 [ 258  100]]


## Očištění a sjednocení signálu

Lorem ipsum dolor sit amet.

In [ ]:
# NEZAPOMENOUT ŘEŠIT OBA KANÁLY

# Sjednocení
## Všechna trénovací data mají stejný sample_rate(44100) i délku(3s) i data_shape(132299, 2), takže není třeba sjednocovat.

# Očištění
# Audio je kvalitní a žádné závažné problémy, které by bylo třeba odstranit

## Amplitude envelope

Lorem ipsum dolor sit amet.

## Spectogram

Lorem ipsum dolor sit amet.

## Neural network

Lorem ipsum dolor sit amet.